In [1]:
def get_web_data(dom = None, cookies = None, headers = None):
    """
       获取每页评论数据
    """
    names = dom.xpath('//div[@class="comment-item "]//span[@class="comment-info"]/a/text()')         # 用户名
    ratings = dom.xpath('//div[@class="comment-item "]//span[@class="comment-info"]/span[2]/@class') # 评分
    times = dom.xpath('//div[@class="comment-item "]//span[@class="comment-info"]/span[@class="comment-time "]/@title') # 评论发布时间
    message = dom.xpath('//div[@class="comment-item "]//div[@class="comment"]//span[@class="short"]/text()')  # 短评正文
    user_url = dom.xpath('//div[@class="comment-item "]//span[@class="comment-info"]/a/@href')  # 用户主页网址
    votes = dom.xpath('//div[@class="comment-item "]//div[@class="comment"]//span[@class="votes vote-count"]/text()') # 赞同数量
    cities = []
    load_times = []
    for i in user_url:
        web_data = requests.get(i, cookies = cookies, headers = header)
        dom_url = etree.HTML(web_data.text, etree.HTMLParser(encoding='utf-8'))
        address = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/a/text()') # 用户居住地
        load_time = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/div[@class="pl"]/text()') # 用户入会时间
        cities.append(address)
        load_times.append(load_time)
        time.sleep(2)
    ratings = ['' if 'rating' not in i else int(re.findall('\d{2}', i)[0]) for i in ratings]  # 评分数据整理
    load_times = ['' if i == [] else i[1].strip()[:-2] for i in load_times]  # 入会数据整理
    cities = ['' if i == [] else i[0] for i in cities]
    data = pd.DataFrame({
        '用户名': names,
        '居住城市': cities,
        '加入时间': load_times,
        '评分': ratings,
        '发表时间': times,
        '短评正文': message,
        '赞同数量': votes
    })
    return data

In [2]:
from selenium import webdriver
from lxml import etree
import requests
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [3]:
driver = webdriver.Chrome()   # 启动chrome浏览器
url = 'https://movie.douban.com/subject/26266893/comments?Status=P'
driver.get(url)   # 获取网页源码数据

In [4]:
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
        }       # 设置请求头

In [5]:
cookies_str ='bid=ruFeVtf87PQ; dbcl2="201478720:PcdzsQpBLy4"; ck=YrrJ; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1609233543%2C%22https%3A%2F%2Faccounts.douban.com%2Fpassport%2Flogin%3Fredir%3Dhttps%253A%252F%252Fmovie.douban.com%252Fsubject%252F26266893%252Fcomments%253Fstatus%253DP%22%5D; _pk_ses.100001.4cf6=*; __utma=30149280.1926750886.1609233544.1609233544.1609233544.1; __utmb=30149280.0.10.1609233544; __utmc=30149280; __utmz=30149280.1609233544.1.1.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/passport/login; __utma=223695111.198307051.1609233544.1609233544.1609233544.1; __utmb=223695111.0.10.1609233544; __utmc=223695111; __utmz=223695111.1609233544.1.1.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/passport/login; push_noty_num=0; push_doumail_num=0; _pk_id.100001.4cf6=c5b140618a1ae23e.1609233543.1.1609233570.1609233543.'
cookies = {}
for i in cookies_str.split(';'):  # 将cookies整理成所需格式
    k, v = i.split('=', 1)
    cookies[k] = v

In [ ]:
# 对所有页面进行数据爬取及解析操作，并进行数据保存
all_data = pd.DataFrame()
wait = WebDriverWait(driver, 20)
while True:
    
    wait.until(  
        EC.element_to_be_clickable(   # 通过该项条件确认网页是否已经加载进来
            (By.CSS_SELECTOR, '#comments > div:nth-child(20) > div.comment > h3 > span.comment-info > a') # 当前页最后一个“用户”按钮是否可以点击
        )
    )
    
    dom = etree.HTML(driver.page_source, etree.HTMLParser(encoding='utf-8'))   # 网页源码解析，得到一个dom文件
    data = get_web_data(dom=dom, cookies=cookies, headers = header)
    all_data = pd.concat([all_data, data], axis=0)
   
    if driver.find_element_by_css_selector('#paginator > a.next')==[]:   # 判定是否还有“后页”按钮
        break
    
    confirm_bnt = wait.until(
        EC.element_to_be_clickable(
            (By.CSS_SELECTOR, '#paginator > a.next')
        )
    )
    confirm_bnt.click()   # 执行翻页操作
    

<selenium.webdriver.remote.webelement.WebElement (session="f0788375369078e261065684dfe30b86", element="f351d7c6-13ea-499f-a19e-830c82b99ac9")>
<class 'selenium.webdriver.remote.webelement.WebElement'>
<selenium.webdriver.remote.webelement.WebElement (session="f0788375369078e261065684dfe30b86", element="c47e150c-7793-48a6-96e5-2f65804eff45")>
<class 'selenium.webdriver.remote.webelement.WebElement'>
<selenium.webdriver.remote.webelement.WebElement (session="f0788375369078e261065684dfe30b86", element="b2263fb4-e3e0-4eae-9c3e-57472affe5ab")>
<class 'selenium.webdriver.remote.webelement.WebElement'>
<selenium.webdriver.remote.webelement.WebElement (session="f0788375369078e261065684dfe30b86", element="7a8abe7c-d348-4b5b-85b4-81404306fe65")>
<class 'selenium.webdriver.remote.webelement.WebElement'>
<selenium.webdriver.remote.webelement.WebElement (session="f0788375369078e261065684dfe30b86", element="4a291ea5-dc6d-428f-88c2-e42fa9e22770")>
<class 'selenium.webdriver.remote.webelement.WebEleme

In [14]:
all_data.to_csv('douban111.csv', index=None, encoding='GB18030')   # 将数据以csv文件的方式写出

In [19]:
all_data.shape

(500, 7)